In [ ]:
#数据选取
import tushare as ts
import pandas as pd
import numpy as np
# xshg_df=ts.get_hist_data('sh')
# xshg_df.sort_index(ascending=True,inplace=True)
import fix_yahoo_finance as yf
xshg_df = yf.download('GLD','2008-01-01','2018-7-31')
xshg_df.drop(columns=['Adj Close'],inplace=True)

xshg_df.rename(columns={'Close':'close'},inplace=True)

xshg_df['p']=xshg_df['close']-xshg_df['close'].shift(1) #这里计算的是当天的波动大小
xshg_df['p']=xshg_df['p'].shift(-1)                     #20天后的波动大小
# xshg_df.dropna(inplace=True)


In [ ]:
def stock_lstm(self,context):

    tf.reset_default_graph() 
    """
    使用LSTM处理股票数据
    直接预测收益
    """
    #获取当前python文件运行在哪个目录下并去掉最后的文件名，如：F:/deeplearning/main.py --> F:/deeplearning
    #在linux下同样起作用
#     basic_path = os.path.dirname(os.path.abspath(__file__))
    #定义存储模型的文件路径
#     os.path.join(basic_path, "stock_rnn_save.ckpt")#表示在运行的python文件路径下保存，文件名为stock_rnn.ckpt，在我环境下运行总是提示“另一个程序正在使用此文件，进程无法访问”，换到其他路径就OK
    # model_save_path = "LSTM_net//stock_rnn_save.ckpt"
    #os.path.join(basic_path, "stock_rnn_save.ckpt")
    #定义训练集的文件路径，当前为运行的python文件路径下，文件名为train_data.csv

    #学习率
    context.learning_rate = 0.001
    #喂数据给LSTM的原始数据有几行，即：一次希望LSTM能“看到”多少个交易日的数据
    context.origin_data_row = 144
    #喂给LSTM的原始数据有几列，即：日线数据有几个元素
    context.origin_data_col = 4
    #LSTM网络有几层
    context.layer_num = 2
    #LSTM网络，每层有几个神经元
    context.cell_num = 256
    #最后输出的数据维度，即：要预测几个数据，该处只预测收益率，只有一个数据
    context.output_num = 1
    #每次给LSTM网络喂多少行经过处理的股票数据。该参数依据自己显卡和网络大小动态调整，越大 一次处理的就越多，越能占用更多的计算资源
    context.batch_size = tf.placeholder(tf.int32, [])
    #输入层、输出层权重、偏置。
    #通过这两对参数，LSTM层能够匹配输入和输出的数据
    context.W = {
     'in':tf.Variable(tf.truncated_normal([context.origin_data_col, context.cell_num], stddev = 1), dtype = tf.float32),
     'out':tf.Variable(tf.truncated_normal([context.cell_num, context.output_num], stddev = 1), dtype = tf.float32)
     }
    context.bias = {
    'in':tf.Variable(tf.constant(0.1, shape=[context.cell_num,]), dtype = tf.float32),
    'out':tf.Variable(tf.constant(0.1, shape=[context.output_num,]), dtype = tf.float32)
    }
    
    #告诉LSTM网络，即将要喂的数据是几行几列
    #None的意思就是喂数据时，行数不确定交给tf自动匹配
    #我们喂得数据行数其实就是batch_size，但是因为None这个位置tf只接受数字变量，而batch_size是placeholder定义的Tensor变量，表示我们在喂数据的时候才会告诉tf具体的值是多少     input_x = tf.placeholder(tf.float32, [None, origin_data_col * origin_data_row])
    context.input_x = tf.placeholder(tf.float32, [None, context.origin_data_col*context.origin_data_row])
    context.input_y = tf.placeholder(tf.float32, [None, context.output_num])
    #处理过拟合问题。该值在其起作用的层上，给该层每一个神经元添加一个“开关”，“开关”打开的概率是keep_prob定义的值，一旦开关被关了，这个神经元的输出将被“阻断”。这样做可以平衡各个神经元起作用的重要性，杜绝某一个神经元“一家独大”，各种大佬都证明这种方法可以有效减弱过拟合的风险。
    context.keep_prob = tf.placeholder(tf.float32, [])

    #通过reshape将输入的input_x转化成2维，-1表示函数自己判断该是多少行，列必须是origin_data_col
    #转化成2维 是因为即将要做矩阵乘法，矩阵一般都是2维的（反正我没见过3维的）
    context.input_x_after_reshape_2 = tf.reshape(context.input_x, [-1, context.origin_data_col])

    #当前计算的这一行，就是输入层。输入层的激活函数是relu,并且施加一个“开关”，其打开的概率为keep_prob
    #input_rnn即是输入层的输出，也是下一层--LSTM层的输入
    context.input_rnn = tf.nn.dropout(tf.nn.relu_layer(context.input_x_after_reshape_2, context.W['in'], context.bias['in']), context.keep_prob)

    #通过reshape将输入的input_rnn转化成3维
    #转化成3维，是因为即将要进入LSTM层，接收3个维度的数据。粗糙点说，即LSTM接受：batch_size个，origin_data_row行cell_num列的矩阵，这里写-1的原因与input_x写None一致
    context.input_rnn = tf.reshape(context.input_rnn, [-1, context.origin_data_row, context.cell_num])

    #定义一个带着“开关”的LSTM单层，一般管它叫细胞
    def lstm_cell():
        cell = rnn.LSTMCell(context.cell_num, reuse = tf.get_variable_scope().reuse)
        return rnn.DropoutWrapper(cell, output_keep_prob = context.keep_prob)
    #这一行就是tensorflow定义多层LSTM网络的代码
    context.lstm_layers = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(context.layer_num)], state_is_tuple = True)
    #初始化LSTM网络
    context.init_state = context.lstm_layers.zero_state(context.batch_size, dtype = tf.float32)

    #使用dynamic_rnn函数，告知tf构建多层LSTM网络，并定义该层的输出
    context.outputs, context.state = tf.nn.dynamic_rnn(context.lstm_layers, inputs = context.input_rnn, initial_state = context.init_state, time_major = False)
    context.h_state =context.state[-1][1]

    #该行代码表示了输出层
    #将LSTM层的输出，输入到输出层，输出层最终得出的值即为预测的收益
    context.y_pre = tf.matmul(context.h_state, context.W['out']) + context.bias['out']

    #损失函数，用作指导tf
    #loss的定义为：(使用预测的收益 - 喂给LSTM的这60个交易日对应的真实收益)的平方，如果有多个（即：batch_size个且batch_size大于1），那就求一下平均值（先挨个求平方，再整个求平均值）
    context.loss = tf.reduce_mean(tf.square(tf.subtract(context.y_pre, context.input_y)))
    #告诉tf，它需要做的事情就是就是尽可能将loss减小
    #learning_rate是减小的这个过程中的参数。如果将我们的目标比喻为“从北向南走路走到菜市场”，我理解的是
    #learning_rate越大，我们走的每一步就迈的越大。初看似乎步子越大越好，但是其实并不能保证每一步都是向南走
    #的，有可能因为训练数据的原因，导致我们朝西走了一大步。或者我们马上就要到菜市场了，但是一大步走过去，给
    #走过了。。。综上，这个learning_rate（学习率参数）的取值，无法给出一个比较普适的，还是需要根据实际情况去
    #尝试和调整。0.001的取值是tf给的默认值
    #上述例子是个人理解用尽可能通俗易懂地语言表达。如有错误，欢迎指正
    context.train_op = tf.train.AdamOptimizer(context.learning_rate).minimize(context.loss)

    #这块定义了一个新的值，用作展示训练的效果
    #它的定义为：选择预测值和实际值差别最大的情况并将差值返回
    context.accuracy = tf.reduce_max(tf.abs(tf.subtract(context.y_pre, context.input_y)))

    #tf要求必须如此定义一个init变量，用以在初始化运行（也就是没有保存模型）时加载各个变量
    init = tf.global_variables_initializer()

    #用以保存参数的函数（跑完下次再跑，就可以直接读取上次跑的结果而不必重头开始）
    saver = tf.train.Saver(write_version=tf.train.SaverDef.V1)
    #获取数据（这是我们自己定义的类）
#     data_get = get_stock_data()
    #设置GPU按需增长，在多个GPU时，能更多地占用资源
    # config = tf.ConfigProto()
    # config.gpu_options.allow_growth = True
    #使用with，保证执行完后正常关闭tf
    context.sess=tf.Session()
    # try:
    #     #定义了存储模型的文件路径，即：当前运行的python文件路径下，文件名为stock_rnn.ckpt
    #     # saver.restore(sess, folder.getTmpDir()+"/model.ckpt") 
    #     print ("成功加载模型参数")
    # except:
    #     #如果是第一次运行，通过init告知tf加载并初始化变量
    #     print ("未加载模型参数，文件被删除或者第一次运行")
    context.sess.run(init)

    #给batch_size赋值
    context._batch_size = 1
    for i in range(10000):
        try:
            #读取训练集数据
#                 train_x, train_y = data_get.get_train_test_data_new(batch_size = _batch_size, file_path = train_csv_path)
            train_x = context.trainX
            train_y = context.trainY
        except StopIteration:
            print ("训练集均已训练完毕")
            context.train_accuracy = context.sess.run(accuracy, feed_dict={
                context.input_x:train_x, context.input_y: train_y, context.keep_prob: 1.0, context.batch_size: context._batch_size})
            print ("step: {0}, training_accuracy: {1}".format(i + 1, context.train_accuracy))
            # saver.save(sess, folder.getTmpDir()+"/model.ckpt") 
            print("保存模型\n")
            break

        if (i + 1) % 20 == 0:
            context.train_accuracy = context.sess.run(context.accuracy, feed_dict={
                context.input_x:train_x, context.input_y: train_y, context.keep_prob: 1.0, context.batch_size: context._batch_size})
            #输出
            print ("step: {0}, training_accuracy: {1}".format(i + 1, context.train_accuracy))
            # saver.save(sess, folder.getTmpDir()+"/model.ckpt") 
            print("保存模型\n")
            ############################################
            #这部分代码作用为：每次保存模型，顺便将预测收益和真实收益输出保存至show_y_pre.txt文件下。熟悉tf可视化，完全可以采用可视化替代
            # _y_pre_train = context.sess.run(y_pre, feed_dict={
            #     input_x: train_x, input_y: train_y, keep_prob: 1.0, batch_size:  _batch_size})
            # _loss = context.sess.run(loss, feed_dict={
            #     input_x:train_x, input_y: train_y, keep_prob: 1.0, batch_size: _batch_size})
            # a1 = np.array(train_y).reshape(1, _batch_size)
            # b1 = np.array(_y_pre_train).reshape(1, _batch_size)
            #按照给定的参数训练一次LSTM神经网络
            context.sess.run(context.train_op, feed_dict={context.input_x: train_x, context.input_y: train_y, context.keep_prob: 0.6, context.batch_size: context._batch_size})